In [1]:
import geopandas as gpd
#导入TransBigData包
import transbigdata as tbd
#读取数据
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
path = "bus_vehicleNum/"
filename = ["01S0D034.txt","01S0D039.txt","01S0D040.txt","01S0D041.txt","01S0D042.txt","01S0D045.txt","01S0D047.txt","01S0D048.txt","01S0D105.txt","01S0D106.txt"]

In [22]:
bounds = [121.4,31.15,122.0,31.35]
params = tbd.area_to_params(bounds, accuracy=500, method='rect')
#bus_load = []
i = 0
for file in filename:
    data = pd.read_csv(path+file)  
    data = tbd.clean_outofbounds(data,bounds = bounds,col = ['lng','lat'])
    data['LONCOL'], data['LATCOL'] = tbd.GPS_to_grid(data['lng'], data['lat'], params)
    time = pd.to_datetime(data['time'])
    data = data.set_index(time)
    dataset = pd.DataFrame(data.resample('5T'))
    bus_map = []
    for row in dataset.itertuples(): 
        a = row[2].groupby(['LONCOL', 'LATCOL'])['vehicleNum'].count().reset_index()
        if not a.empty: 
            pre = row
            a = a.iloc[0]
        else: 
            a = pre[2].groupby(['LONCOL', 'LATCOL'])['vehicleNum'].count().reset_index()
            a = a.iloc[0]
        a['time'] = row[1]
        bus_map.append(a)
    bus = pd.DataFrame(bus_map)
    bus['geometry'] = tbd.grid_to_polygon([bus['LONCOL'], bus['LATCOL']],params)
    bus = gpd.GeoDataFrame(bus)
    bus.to_csv("bus_" + str(i) + ".csv",index=False)
    i+=1
    #bus_load.append(bus)
    

In [27]:
i = 0
bus_dd = []
path2 = "bus_"
while i< 10:
    bus_dd.append(pd.read_csv(path2 + str(i) + ".csv"))
    i += 1
bus_dd

[      LONCOL  LATCOL  vehicleNum                 time  \
 0         19      28           1  2007-02-19 08:40:00   
 1         19      27           1  2007-02-19 08:45:00   
 2         19      27           1  2007-02-19 08:50:00   
 3         19      27           1  2007-02-19 08:55:00   
 4         19      27           1  2007-02-19 09:00:00   
 ...      ...     ...         ...                  ...   
 1331      15      26           1  2007-02-23 23:35:00   
 1332      18      25           5  2007-02-23 23:40:00   
 1333      19      25           1  2007-02-23 23:45:00   
 1334      19      27           1  2007-02-23 23:50:00   
 1335      19      28           1  2007-02-23 23:55:00   
 
                                                geometry  
 0     POLYGON ((121.49730499619025 31.27365664317663...  
 1     POLYGON ((121.49730499619025 31.26916003797020...  
 2     POLYGON ((121.49730499619025 31.26916003797020...  
 3     POLYGON ((121.49730499619025 31.26916003797020...  
 4     

In [19]:
for bus_bum in bus_dd:
    

D:\Anaconda\envs\gym\lib\site-packages\pandas\core\internals\construction.py:576: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  values = np.array([convert(v) for v in values])


,0
0,LONCOL LATCOL vehicleNum ...
1,LONCOL LATCOL vehicleNum ...
2,LONCOL LATCOL vehicleNum ...
3,LONCOL LATCOL vehicleNum ...
4,LONCOL LATCOL vehicleNum ...
5,LONCOL LATCOL vehicleNum ...
6,LONCOL LATCOL vehicleNum ...
7,LONCOL LATCOL vehicleNum ...
8,LONCOL LATCOL vehicleNum ...
9,LONCOL LATCOL vehicleNum ...


In [38]:
bus_dd[0].iloc[0]

LONCOL                                                       19
LATCOL                                                       28
vehicleNum                                                    1
time                                        2007-02-19 08:40:00
geometry      POLYGON ((121.49730499619025 31.27365664317663...
Name: 0, dtype: object